# Universidad del Valle de Guatemala
## Facultad de Ingeniería
### Departamento de Computación

---

# Proyecto 1: Establecimientos educativos de Guatemala
## Avances del proyecto 1. Limpieza de datos - Dataset: Establecimientos educativos de Guatemala

**Integrantes:**
- Diego Alexander Hernández Silvestre, 21270
- Linda Inés Jiménez Vides, 21169
- Mario Antonio Guerra Morales, 21008
- David Jonathan Aragon Vasquez, 21053

**Curso:** Data Science  
**Sección:** 10  
**Grupo** 4  

---

Guatemala, 25 de julio de 2024


In [23]:
# Importar las librerías necesarias
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from bs4 import BeautifulSoup
from ucimlrepo import fetch_ucirepo 
from pydantic_settings import BaseSettings
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from tabulate import tabulate

## 📜 **Conversión de HTML y XLS a CSV**

1. **🔍 Buscar Tablas en HTML**: 
   - Convierte el contenido HTML con tablas a formato CSV. 🗂️

2. **🔄 Procesar Archivos**: 
   - Lee archivos `.xls` y convierte datos a HTML para luego a CSV. 🗃️

3. **🔄 Convertir a CSV**: 
   - Convierte archivos `.xls` en un directorio y sus subdirectorios a CSV. 📁

4. **🚫 Manejo de Errores**: 
   - Maneja errores de lectura y conversión para archivos corruptos. ⚠️


In [24]:
def convert_html_table_to_csv(html_content, csv_path):
    soup = BeautifulSoup(html_content, 'html.parser')
    tables = soup.find_all('table')
    
    if not tables:
        raise ValueError("No se encontró ninguna tabla en el contenido HTML.")
    
    for table in tables:
        headers = [th.get_text(strip=True) for th in table.find_all('th')]
        if not headers:
            headers = [td.get_text(strip=True) for td in table.find_all('tr')[0].find_all('td')]

        rows = []
        for row in table.find_all('tr')[1:]:
            cells = [td.get_text(strip=True) for td in row.find_all('td')]
            if cells:
                rows.append(cells)

        if headers and rows:
            try:
                df = pd.DataFrame(rows, columns=headers)
                df.to_csv(csv_path, index=False, quoting=csv.QUOTE_NONNUMERIC)
                print(f"Convertido HTML a CSV en {csv_path}")
                break  # Salir del bucle si la tabla se convirtió correctamente
            except Exception as e:
                print(f"Error al crear el CSV para {csv_path}: {e}")
                continue

def process_file(file_path, csv_path):
    try:
        # Intentar leer el archivo como texto bruto y tratarlo como HTML
        with open(file_path, 'r', encoding='latin1') as file:
            html_content = file.read()
            convert_html_table_to_csv(html_content, csv_path)
    except (UnicodeDecodeError, IOError):
        try:
            # Leer el archivo .xls usando pandas si es un archivo Excel válido
            # Nota: Esto puede fallar si el archivo está corrupto
            try:
                df = pd.read_excel(file_path, engine='openpyxl')
                html_content = df.to_html(index=False)
                convert_html_table_to_csv(html_content, csv_path)
            except Exception as e:
                print(f"No se pudo leer el archivo Excel {file_path}: {e}")
        except Exception as e:
            print(f"No se pudo procesar {file_path}: {e}")

def convert_xls_to_csv(root_directory):
    for subdir, _, files in os.walk(root_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(subdir, file)
                csv_filename = file.replace(".xls", ".csv")
                csv_path = os.path.join(subdir, csv_filename)
                process_file(file_path, csv_path)

# Especifica el directorio raíz que contiene los archivos .xls y subdirectorios
root_directory = "Data/"

convert_xls_to_csv(root_directory)


Error al crear el CSV para Data/ALTA_VERAPAZ\establecimiento.csv: 6393 columns passed, passed data had 17 columns
Convertido HTML a CSV en Data/ALTA_VERAPAZ\establecimiento.csv
Error al crear el CSV para Data/BAJA_VERAPAZ\establecimiento.csv: 1973 columns passed, passed data had 17 columns
Convertido HTML a CSV en Data/BAJA_VERAPAZ\establecimiento.csv
Error al crear el CSV para Data/CHIMALTENANGO\establecimiento.csv: 6138 columns passed, passed data had 17 columns
Convertido HTML a CSV en Data/CHIMALTENANGO\establecimiento.csv
Error al crear el CSV para Data/CHIQUIMULA\establecimiento.csv: 2925 columns passed, passed data had 17 columns
Convertido HTML a CSV en Data/CHIQUIMULA\establecimiento.csv
Error al crear el CSV para Data/CIUDAD_CAPITAL\establecimiento.csv: 26623 columns passed, passed data had 17 columns
Convertido HTML a CSV en Data/CIUDAD_CAPITAL\establecimiento.csv
Error al crear el CSV para Data/EL_PROGRESO\establecimiento.csv: 2160 columns passed, passed data had 17 columns

## 🗑️ **Eliminación de Filas Vacías en CSV**

1. **📥 Leer CSV**: 
   - Lee archivos CSV desde el directorio especificado. 📄

2. **🧹 Limpiar Datos**: 
   - Elimina filas donde todas las celdas están vacías. 🚮

3. **📈 Reportar Cambios**: 
   - Guarda el archivo limpio y muestra cuántas filas fueron eliminadas. 📊

4. **🔄 Procesar Todos los CSV**: 
   - Procesa todos los archivos CSV en el directorio, exceptuando los ya procesados. 📁


In [25]:
def remove_empty_rows_from_csv(csv_path, processed_csv_path):
    try:
        # Leer el archivo CSV
        df = pd.read_csv(csv_path)

        # Contar el número de filas originales
        original_row_count = len(df)

        # Eliminar filas donde todas las celdas están vacías
        df_cleaned = df.dropna(how='all')

        # Contar el número de filas después de eliminar las vacías
        cleaned_row_count = len(df_cleaned)

        # Contar cuántas filas fueron eliminadas
        rows_removed = original_row_count - cleaned_row_count

        # Guardar el DataFrame limpio en un nuevo archivo CSV
        df_cleaned.to_csv(processed_csv_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

        # Informar cuántas filas fueron eliminadas
        print(f"Archivo procesado guardado en {processed_csv_path}. Filas eliminadas: {rows_removed}")

    except Exception as e:
        print(f"No se pudo procesar el archivo {csv_path}: {e}")

def process_all_csv_files(root_directory):
    for subdir, _, files in os.walk(root_directory):
        for file in files:
            if file.endswith(".csv") and not file.endswith("processed.csv"):
                csv_path = os.path.join(subdir, file)
                processed_csv_path = os.path.join(subdir, file.replace(".csv", "_processed.csv"))
                remove_empty_rows_from_csv(csv_path, processed_csv_path)

# Especifica el directorio raíz que contiene los archivos CSV
root_directory = "Data/"

process_all_csv_files(root_directory)


Archivo procesado guardado en Data/ALTA_VERAPAZ\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/BAJA_VERAPAZ\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/CHIMALTENANGO\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/CHIQUIMULA\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/CIUDAD_CAPITAL\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/EL_PROGRESO\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/ESCUINTLA\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/GUATEMALA\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/HUEHUETENANGO\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado guardado en Data/IZABAL\establecimiento_processed.csv. Filas eliminadas: 1
Archivo procesado gua